<a href="https://colab.research.google.com/github/Stefi96/DetectingNFTs-Master/blob/main/Domains_NFT_Master.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# @title Lets import the libraries and code
import pandas as pd

In [2]:
# Load the newly uploaded datasets

In [3]:
scam_data = pd.read_csv('/content/drive/MyDrive/Master/Project/Domains/final_blocklist_domains.csv')
legit_data = pd.read_csv('/content/drive/MyDrive/Master/Project/Domains/final_legit_domains.csv')

In [4]:
# Display the first few rows of each dataset for verification
scam_data, legit_data

(                               url
 0              blogpost-opensea.io
 1                   phantomweb.app
 2                       aurory.app
 3                     solvision.io
 4                    staratlas.art
 ...                            ...
 1625  defi-1690929004.devflow.best
 1626   defi-offer-s008.netlify.app
 1627   defi-offer-s009.netlify.app
 1628         markets-looksrare.com
 1629            looksrare-org.info
 
 [1630 rows x 1 columns],
                Unique Domains
 0               basis.markets
 1            skeletoncrew.rip
 2                flippies.art
 3    meerkatmillionaires.club
 4             stylishstuds.io
 ..                        ...
 816                koinkoi.io
 817           burritoboyz.wtf
 818                  t00b.app
 819           outcast.academy
 820   apesofwallstreetnft.com
 
 [821 rows x 1 columns])

In [ ]:
# Labeling the datasets
scam_data['label'] = 1
legit_data['label'] = 0

# Renaming columns for consistency
scam_data.rename(columns={'url': 'domain'}, inplace=True)
legit_data.rename(columns={'Unique Domains': 'domain'}, inplace=True)

# Combining the datasets
combined_dataset = pd.concat([scam_data, legit_data], axis=0).reset_index(drop=True)

# Display the first few rows of the combined dataset
combined_dataset

,domain,label,url
0,blogpost-opensea.io,1,NaN
1,phantomweb.app,1,NaN
2,aurory.app,1,NaN
3,solvision.io,1,NaN
4,staratlas.art,1,NaN
...,...,...,...
2446,NaN,0,koinkoi.io
2447,NaN,0,burritoboyz.wtf
2448,NaN,0,t00b.app
2449,NaN,0,outcast.academy


In [ ]:
# import lightgbm as lgb

# # Initializing the LightGBM model
# lgbm_model = lgb.LGBMClassifier(objective='binary', class_weight='balanced', random_state=42)

# # Training the model
# lgbm_model.fit(X_train_engineered, y_train_engineered)

# # Predictions
# y_pred_lgbm = lgbm_model.predict(X_test_engineered)

# # Evaluation
# accuracy_lgbm = accuracy_score(y_test_engineered, y_pred_lgbm)
# precision_lgbm = precision_score(y_test_engineered, y_pred_lgbm)
# recall_lgbm = recall_score(y_test_engineered, y_pred_lgbm)
# f1_lgbm = f1_score(y_test_engineered, y_pred_lgbm)

# accuracy_lgbm, precision_lgbm, recall_lgbm, f1_lgbm

In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from imblearn.over_sampling import SMOTE

In [ ]:
scam_data['label'] = 1
legit_data['label'] = 0
#combined_data = pd.concat([scam_data, legit_data], axis=0)

# Renaming the 'Unique Domains' column in legit dataset to 'url'
legit_data.rename(columns={"Unique Domains": "url"}, inplace=True)

# Combining the datasets again and assigning labels
combined_data = pd.concat([scam_data, legit_data], axis=0)

# Check for NA
combined_data.isna().sum()

url      1
label    0
dtype: int64

In [ ]:
combined_data['url'].count()
combined_data.dropna()

,url,label,tokenized_url,tokenized_url_str
0,blogpost-opensea.io,1,"[blogpost, opensea, io]",blogpost opensea io
1,phantomweb.app,1,"[phantomweb, app]",phantomweb app
2,aurory.app,1,"[aurory, app]",aurory app
3,solvision.io,1,"[solvision, io]",solvision io
4,staratlas.art,1,"[staratlas, art]",staratlas art
...,...,...,...,...
816,koinkoi.io,0,"[koinkoi, io]",koinkoi io
817,burritoboyz.wtf,0,"[burritoboyz, wtf]",burritoboyz wtf
818,t00b.app,0,"[t00b, app]",t00b app
819,outcast.academy,0,"[outcast, academy]",outcast academy


In [ ]:
# NLP preprocessing
#combined_data['url'].fillna("", inplace=True)
#combined_data['url'] = combined_data['url'].fillna("missing")
combined_data['tokenized_url'] = combined_data['url'].str.split(r'\W+')
combined_data['tokenized_url_str'] = combined_data['tokenized_url'].str.join(' ')
vectorizer = TfidfVectorizer(ngram_range=(1, 2))
X_tfidf = vectorizer.fit_transform(combined_data['tokenized_url_str'])
X_train_tfidf, X_test_tfidf, y_train_tfidf, y_test_tfidf = train_test_split(X_tfidf, combined_data['label'], test_size=0.3, random_state=42)


ValueError: ignored

In [ ]:
nan_string_count = combined_data[combined_data['url'] == ""].shape[0]
nan_string_count

1

In [ ]:
# Random Forest with TF-IDF
rf_clf_tfidf = RandomForestClassifier(n_estimators=100, random_state=42)
rf_clf_tfidf.fit(X_train_tfidf, y_train_tfidf)
y_pred_rf_tfidf = rf_clf_tfidf.predict(X_test_tfidf)
accuracy_rf_tfidf = accuracy_score(y_test_tfidf, y_pred_rf_tfidf)
precision_rf_tfidf = precision_score(y_test_tfidf, y_pred_rf_tfidf)
recall_rf_tfidf = recall_score(y_test_tfidf, y_pred_rf_tfidf)
f1_rf_tfidf = f1_score(y_test_tfidf, y_pred_rf_tfidf)

print("Random Forest with TF-IDF Metrics:")
print("Accuracy:", accuracy_rf_tfidf)
print("Precision:", precision_rf_tfidf)
print("Recall:", recall_rf_tfidf)
print("F1-Score:", f1_rf_tfidf)

Random Forest with TF-IDF Metrics:
Accuracy: 0.7758152173913043
Precision: 0.7682737169517885
Recall: 0.9686274509803922
F1-Score: 0.8568950563746747


In [ ]:
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train_tfidf, y_train_tfidf)

# Training Random Forest on the SMOTE balanced data
rf_clf_smote = RandomForestClassifier(n_estimators=100, random_state=42)
rf_clf_smote.fit(X_train_smote, y_train_smote)

# Predictions
y_pred_rf_smote = rf_clf_smote.predict(X_test_tfidf)

# Evaluation
accuracy_rf_smote = accuracy_score(y_test_tfidf, y_pred_rf_smote)
precision_rf_smote = precision_score(y_test_tfidf, y_pred_rf_smote)
recall_rf_smote = recall_score(y_test_tfidf, y_pred_rf_smote)
f1_rf_smote = f1_score(y_test_tfidf, y_pred_rf_smote)

accuracy_rf_smote, precision_rf_smote, recall_rf_smote, f1_rf_smote

(0.7744565217391305, 0.76875, 0.9647058823529412, 0.8556521739130436)

In [ ]:
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier
bagging_clf = BaggingClassifier(base_estimator=RandomForestClassifier(n_estimators=100, random_state=42),
                                n_estimators=10, random_state=42)
bagging_clf.fit(X_train_smote, y_train_smote)

# Predictions
y_pred_bagging = bagging_clf.predict(X_test_tfidf)

# Evaluation
accuracy_bagging = accuracy_score(y_test_tfidf, y_pred_bagging)
precision_bagging = precision_score(y_test_tfidf, y_pred_bagging)
recall_bagging = recall_score(y_test_tfidf, y_pred_bagging)
f1_bagging = f1_score(y_test_tfidf, y_pred_bagging)

print("Accuracy:", accuracy_bagging)
print("Precision:", precision_bagging)
print("Recall:", recall_bagging)
print("F1-Score:", f1_bagging)

/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


Accuracy: 0.7730978260869565
Precision: 0.7675507020280812
Recall: 0.9647058823529412
F1-Score: 0.8549087749782799


In [ ]:
nan_string_count = combined_data[combined_data['url'] == ""].shape[0]
nan_string_count

1

In [ ]:
combined_data

,url,label,tokenized_url,tokenized_url_str
0,blogpost-opensea.io,1,"[blogpost, opensea, io]",blogpost opensea io
1,phantomweb.app,1,"[phantomweb, app]",phantomweb app
2,aurory.app,1,"[aurory, app]",aurory app
3,solvision.io,1,"[solvision, io]",solvision io
4,staratlas.art,1,"[staratlas, art]",staratlas art
...,...,...,...,...
816,koinkoi.io,0,"[koinkoi, io]",koinkoi io
817,burritoboyz.wtf,0,"[burritoboyz, wtf]",burritoboyz wtf
818,t00b.app,0,"[t00b, app]",t00b app
819,outcast.academy,0,"[outcast, academy]",outcast academy


In [ ]:
bagging_clf = BaggingClassifier(base_estimator=RandomForestClassifier(n_estimators=100, random_state=42),
                                n_estimators=10, random_state=42)
bagging_clf.fit(X_train_smote, y_train_smote)

# Predictions
y_pred_bagging = bagging_clf.predict(X_test_tfidf)

# Evaluation
accuracy_bagging = accuracy_score(y_test_tfidf, y_pred_bagging)
precision_bagging = precision_score(y_test_tfidf, y_pred_bagging)
recall_bagging = recall_score(y_test_tfidf, y_pred_bagging)
f1_bagging = f1_score(y_test_tfidf, y_pred_bagging)

print("Accuracy:", accuracy_bagging)
print("Precision:", precision_bagging)
print("Recall:", recall_bagging)
print("F1-Score:", f1_bagging)

/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


Accuracy: 0.7730978260869565
Precision: 0.7675507020280812
Recall: 0.9647058823529412
F1-Score: 0.8549087749782799


In [ ]:
nan_string_count = combined_data[combined_data['url'] == ""].shape[0]
nan_string_count

1

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Load and merge data
scam_data = pd.read_csv('/content/drive/MyDrive/Master/Project/Domains/final_blocklist_domains.csv')
legit_data = pd.read_csv('/content/drive/MyDrive/Master/Project/Domains/final_legit_domains.csv')

scam_data['label'] = 1
legit_data['label'] = 0
#combined_data = pd.concat([scam_data, legit_data], axis=0)

# Renaming the 'Unique Domains' column in legit dataset to 'url'
legit_data.rename(columns={"Unique Domains": "url"}, inplace=True)

# Combining the datasets again and assigning labels
combined_data = pd.concat([scam_data, legit_data], axis=0)
combined_data = combined_data.dropna()

# Check for NaN values in the 'url' column after the correction
nan_count_after_correction = combined_data['url'].isna().sum()
nan_count_after_correction


# Convert the 'url' column to string
combined_data['url'] = combined_data['url'].astype(str)

nan_string_count = combined_data[combined_data['url'] == "nan"].shape[0]
print('NAN test:',nan_string_count)

# Feature engineering
combined_data['domain_length'] = combined_data['url'].apply(lambda x: len(x))
combined_data['num_special_chars'] = combined_data['url'].apply(lambda x: sum([1 for char in x if not char.isalnum()]))
combined_data['num_subdomains'] = combined_data['url'].apply(lambda x: x.count('.'))
combined_data['has_http'] = combined_data['url'].apply(lambda x: 1 if 'http' in x else 0)
combined_data['has_https'] = combined_data['url'].apply(lambda x: 1 if 'https' in x else 0)
combined_data['has_www'] = combined_data['url'].apply(lambda x: 1 if 'www' in x else 0)
combined_data['has_numbers'] = combined_data['url'].apply(lambda x: 1 if any(char.isdigit() for char in x) else 0)

#combined_data

NAN test: 0


In [ ]:
from sklearn.preprocessing import LabelEncoder

# Applying label encoding to the 'url' column
label_encoder = LabelEncoder()
combined_data['url'] = label_encoder.fit_transform(combined_data['url'])

X = combined_data.drop("label", axis=1)
y = combined_data["label"]

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Model training
bagging_clf = BaggingClassifier(base_estimator=RandomForestClassifier(n_estimators=100, random_state=42),
                                n_estimators=10, random_state=42)
bagging_clf.fit(X_train, y_train)

# Model evaluation
y_pred_bagging = bagging_clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred_bagging)
precision = precision_score(y_test, y_pred_bagging)
recall = recall_score(y_test, y_pred_bagging)
f1 = f1_score(y_test, y_pred_bagging)

print(f"Accuracy: {accuracy}, Precision: {precision}, Recall: {recall}, F1-Score: {f1}")

combined_data

/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


Accuracy: 0.7523809523809524, Precision: 0.7954545454545454, Recall: 0.8502024291497976, F1-Score: 0.821917808219178


,url,label,domain_length,num_special_chars,num_subdomains,has_http,has_https,has_www,has_numbers
0,184,1,19,2,1,0,0,0,0
1,1421,1,14,1,1,0,0,0,0
2,133,1,10,1,1,0,0,0,0
3,2100,1,12,1,1,0,0,0,0
4,2129,1,13,1,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...
816,801,0,10,1,1,0,0,0,0
817,267,0,15,1,1,0,0,0,0
818,2179,0,8,1,1,0,0,0,1
819,1336,0,15,1,1,0,0,0,0


In [ ]:
combined_data

,url,label,domain_length,num_special_chars,num_subdomains,has_http,has_https,has_www,has_numbers
0,184,1,19,2,1,0,0,0,0
1,1421,1,14,1,1,0,0,0,0
2,133,1,10,1,1,0,0,0,0
3,2100,1,12,1,1,0,0,0,0
4,2129,1,13,1,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...
816,801,0,10,1,1,0,0,0,0
817,267,0,15,1,1,0,0,0,0
818,2179,0,8,1,1,0,0,0,1
819,1336,0,15,1,1,0,0,0,0


In [ ]:
nan_string_count = combined_data[combined_data['url'] == "nan"].shape[0]
nan_string_count

0

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from xgboost import XGBClassifier

# Load the data
scam_data = pd.read_csv('/content/drive/MyDrive/Master/Project/Domains/final_blocklist_domains.csv')
legit_data = pd.read_csv('/content/drive/MyDrive/Master/Project/Domains/final_legit_domains.csv')

# label the data
scam_data['label'] = 1
legit_data['label'] = 0

# Renaming the 'Unique Domains' column in legit dataset to 'url'
legit_data.rename(columns={"Unique Domains": "url"}, inplace=True)

# Combine the datasets
combined_data = pd.concat([scam_data, legit_data], ignore_index=True)

# Fill NaN with placeholder
combined_data['url'] = combined_data['url'].fillna("missing")

# Label encoding the URLs
label_encoder = LabelEncoder()
combined_data['url_encoded'] = label_encoder.fit_transform(combined_data['url'])

# Extracting features from the URLs
combined_data['domain_length'] = combined_data['url'].apply(lambda x: len(x))
combined_data['num_special_chars'] = combined_data['url'].apply(lambda x: sum([1 for char in x if not char.isalnum()]))
combined_data['num_subdomains'] = combined_data['url'].apply(lambda x: x.count('.'))
combined_data['has_http'] = combined_data['url'].apply(lambda x: 1 if 'http' in x else 0)
combined_data['has_https'] = combined_data['url'].apply(lambda x: 1 if 'https' in x else 0)
combined_data['has_www'] = combined_data['url'].apply(lambda x: 1 if 'www' in x else 0)
combined_data['has_numbers'] = combined_data['url'].apply(lambda x: 1 if any(char.isdigit() for char in x) else 0)


In [3]:
combined_data

,url,label,url_encoded,domain_length,num_special_chars,num_subdomains,has_http,has_https,has_www,has_numbers
0,blogpost-opensea.io,1,184,19,2,1,0,0,0,0
1,phantomweb.app,1,1422,14,1,1,0,0,0,0
2,aurory.app,1,133,10,1,1,0,0,0,0
3,solvision.io,1,2101,12,1,1,0,0,0,0
4,staratlas.art,1,2130,13,1,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
2446,koinkoi.io,0,801,10,1,1,0,0,0,0
2447,burritoboyz.wtf,0,267,15,1,1,0,0,0,0
2448,t00b.app,0,2180,8,1,1,0,0,0,1
2449,outcast.academy,0,1337,15,1,1,0,0,0,0


In [2]:
# NLP transformations on the URLs
tfidf_vectorizer = TfidfVectorizer(max_features=100)

In [3]:
url_tfidf = tfidf_vectorizer.fit_transform(combined_data['url'])

In [4]:
url_tfidf_df = pd.DataFrame(url_tfidf.toarray(), columns=tfidf_vectorizer.get_feature_names_out())

In [5]:
combined_data = pd.concat([combined_data, url_tfidf_df], axis=1)
combined_data.drop(columns=['url'], inplace=True)

In [6]:
combined_data

,label,url_encoded,domain_length,num_special_chars,num_subdomains,has_http,has_https,has_www,has_numbers,alpha,...,uno,v2,web,website,work,world,xyz,y00ts,yoots,zerium
0,1,184,19,2,1,0,0,0,0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,1422,14,1,1,0,0,0,0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,133,10,1,1,0,0,0,0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,2101,12,1,1,0,0,0,0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1,2130,13,1,1,0,0,0,0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2446,0,801,10,1,1,0,0,0,0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2447,0,267,15,1,1,0,0,0,0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2448,0,2180,8,1,1,0,0,0,1,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2449,0,1337,15,1,1,0,0,0,0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
# Splitting the data
X = combined_data.drop("label", axis=1)
y = combined_data["label"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Random Forest
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)
rf_predictions = rf_model.predict(X_test)
rf_metrics = (accuracy_score(y_test, rf_predictions), precision_score(y_test, rf_predictions), recall_score(y_test, rf_predictions), f1_score(y_test, rf_predictions))

rf_metrics


(0.8024439918533605,
 0.8484848484848485,
 0.8562691131498471,
 0.8523592085235921)

In [9]:
# XGBoost
xgb_model = XGBClassifier(random_state=42)
xgb_model.fit(X_train, y_train)
xgb_predictions = xgb_model.predict(X_test)
xgb_metrics = (accuracy_score(y_test, xgb_predictions), precision_score(y_test, xgb_predictions), recall_score(y_test, xgb_predictions), f1_score(y_test, xgb_predictions))

xgb_metrics

(0.845213849287169, 0.8885448916408669, 0.8776758409785933, 0.8830769230769231)

In [ ]:
# SVM
svm_model = SVC(kernel='linear', probability=True, random_state=42)
svm_model.fit(X_train, y_train)
svm_predictions = svm_model.predict(X_test)
svm_metrics = (accuracy_score(y_test, svm_predictions), precision_score(y_test, svm_predictions), recall_score(y_test, svm_predictions), f1_score(y_test, svm_predictions))

svm_metrics

In [ ]:
# Ensemble Method (RF + SVM)
ensemble_predictions = (rf_model.predict_proba(X_test)[:, 1] + svm_model.predict_proba(X_test)[:, 1]) / 2
ensemble_final_predictions = [1 if prob > 0.5 else 0 for prob in ensemble_predictions]

In [ ]:
# Hyperparameter tuning for RandomForest
param_grid = {
    'n_estimators': [50, 100, 150],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

grid_search = GridSearchCV(rf_clf, param_grid, cv=3, scoring='accuracy', verbose=2, n_jobs=-1)
grid_search.fit(X_train, y_train)

best_rf_clf = grid_search.best_estimator_
y_pred_best_rf = best_rf_clf.predict(X_test)

accuracy_best_rf = accuracy_score(y_test, y_pred_best_rf)
precision_best_rf = precision_score(y_test, y_pred_best_rf)
recall_best_rf = recall_score(y_test, y_pred_best_rf)
f1_best_rf = f1_score(y_test, y_pred_best_rf)

print("SVM Results:", svm_metrics)
print("RandomForest Results:", rf_metrics)
print("XGBoost Results:", svm_metrics)
print("Tuned RandomForest Results:", accuracy_best_rf, precision_best_rf, recall_best_rf, f1_best_rf)